In [3]:
import os

print(os.getcwd())

print('CSV files:', [f for f in os.listdir('csv_files') if f.endswith('.csv')])

c:\Users\fadri\Desktop\4. ISC FS\codigos\IFS06-TE\TE_web_dashboard\web_app
CSV files: ['2023-08-10_15-21-12_-_FS-Acceleration_FSS-Miguel_Pollens.csv', 'example1.csv']


In [13]:
CLOUD_ORG = '44b2810a938c40f7'
CLOUD_TOKEN = 'Ncw6Rq19Xt1FXg3JMLS8vqToZT6RON6yRM0JmiBIAx8pZTs5RBth38iPFHu0Wfw6C71sqVEGag_IKcHOx1xyow=='
CLOUD_URL = 'https://us-east-1-1.aws.cloud2.influxdata.com'
CLOUD_URL_BUCKETS = 'https://us-east-1-1.aws.cloud2.influxdata.com/api/v2/buckets'

OSS_ORG = 'b2b03940375e28ac'
OSS_TOKEN = 'sNZJlQsv2KhY28xPDZBmt2HbgGtwU5AE4vUjQsks9WPm3o0g_UreLkxNVgMHNotIqUMHCcgAyN4llUvdXs4QtA=='
OSS_URL = 'http://localhost:8086'

In [24]:
import os
from influxdb_client import InfluxDBClient

def csv_buckets():
    csv_files = [f for f in os.listdir('csv_files') if f.endswith('.csv')]
    return [f.split('.')[0] for f in csv_files]

def buckets_cloud(cloud_url, cloud_token, cloud_org):
    query = 'buckets()'
    cloud_client = InfluxDBClient(url=cloud_url, token=cloud_token, org=cloud_org)
    tables = cloud_client.query_api().query(query, org=cloud_org)
    buckets = []
    for table in tables:
        for record in table.records:
            buckets.append(record.values['name'])

    replaced_buckets = []
    for bucket in buckets:
        bucket_ = bucket.replace(" ", "_").replace(":", "-").replace("|>", "-")
        replaced_buckets.append(bucket_)

    return buckets, replaced_buckets

def download_bucket(bucket_name):
    # Implementa la lógica para descargar el bucket aquí
    print(f"Descargando bucket: {bucket_name}")

def compare_and_download_buckets(cloud_url, cloud_token, cloud_org):
    original_cloud_buckets, replaced_cloud_buckets = buckets_cloud(cloud_url, cloud_token, cloud_org)
    local_buckets = csv_buckets()

    # Encontrar los índices de los buckets en la nube que no están en los archivos CSV locales
    missing_indices = [i for i, bucket in enumerate(replaced_cloud_buckets) if bucket not in local_buckets]

    # Descargar los buckets faltantes usando los índices para obtener los nombres originales
    for index in missing_indices:
        download_bucket(original_cloud_buckets[index])


compare_and_download_buckets(CLOUD_URL, CLOUD_TOKEN, CLOUD_ORG)

Descargando bucket: 2023-07-25 16:41:52 |> FS-jarama2-fadrique
Descargando bucket: 2023-07-25 17:12:00 |> FS-jarama-fad
Descargando bucket: 2023-07-25 17:19:50 |> FS-jarama-test10ms
Descargando bucket: 2023-07-25 17:25:37 |> FS-jarama-test100ms
Descargando bucket: 2023-07-25 17:42:12 |> FS-2-1
Descargando bucket: 2023-07-31 12:23:05 |> FS-Autodromo de Boadilla-Miguel Torrens
Descargando bucket: 2023-07-31 12:29:05 |> FS-Autodromo de Boadilla-Torrens
Descargando bucket: 2023-07-31 12:35:49 |> FS-Monza-Jaime Landa
Descargando bucket: 2023-08-02 13:18:19 |> FS-Autodromo de Boadilla-Jaime Landa
Descargando bucket: 2023-08-05 00:40:47 |> FS-Boadilla-JG
Descargando bucket: 2023-08-05 01:28:53 |> FS-Autodromo de Boadilla-Juan Gracia
Descargando bucket: 2023-08-05 01:57:21 |> FS-Autodromo de Boadilla (ext)-Juan Gracia
Descargando bucket: 2023-08-05 02:14:06 |> FS--Juan Gracia
Descargando bucket: 2023-08-05 03:17:45 |> FS-Autodromo de Boadilla-Juan Gracia
Descargando bucket: 2023-08-06 01:01:41

In [43]:
import csv
from influxdb_client import InfluxDBClient
from time import time
import os

def download_data(selected_bucket, cloud_url, cloud_token, cloud_org):

    # check if the selected bucket contains any of the words in the invalid list
    ignore_buckets = ["test", "ISC", "RPI", "_monitoring", "_tasks", "version_"]

    if any(word in selected_bucket for word in ignore_buckets):
        print(f"Bucket {selected_bucket} is not valid")
        return

    csv_folder = "csv_files"

    if not os.path.exists(csv_folder):
        csv_folder = os.getcwd()


    client = InfluxDBClient(url=cloud_url, token=cloud_token)

    # Estar atento a range, ya que habria que cambiarlo en algun momento
    query = 'from(bucket: "' + selected_bucket + '") |> range(start: -500d)'

    t0 = time()
    result = client.query_api().query(org=cloud_org, query=query)

    print(result)
    print(f'Done in {time() - t0} seconds')

    points = []

    csv_filename = f"{selected_bucket}.csv"
    csv_filename = csv_filename.replace(" ", "_")
    csv_filename = csv_filename.replace(":", "-")
    csv_filename = csv_filename.replace("|>", "-")

    csv_filename = os.path.join(csv_folder, csv_filename)

    for table in result:
        for record in table.records:
            point = {
                "time": record.get_time(),
                "measurement": record.get_field(), 
                "value": record.get_value()
            }
            points.append(point)

    points = [point for point in points if point['measurement'] != 'data']

    print(len(points))

    if len(points) == 0:
        print(f"No data found for bucket {selected_bucket}")
        return

    with open(f'{csv_filename}', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["time", "measurement", "value"])
        writer.writeheader()
        for point in points:
            writer.writerow(point)

    print(f"File {csv_filename} saved in the current directory")

In [40]:
download_data('2023-07-25 16:25:36 |> FS-jarama-fadrique', CLOUD_URL, CLOUD_TOKEN, CLOUD_ORG)

[<FluxTable: 8 columns, 875 records>, <FluxTable: 8 columns, 7 records>, <FluxTable: 8 columns, 15 records>, <FluxTable: 8 columns, 7 records>, <FluxTable: 8 columns, 210 records>, <FluxTable: 8 columns, 79 records>, <FluxTable: 8 columns, 24 records>, <FluxTable: 8 columns, 18 records>, <FluxTable: 8 columns, 16 records>, <FluxTable: 8 columns, 2 records>, <FluxTable: 8 columns, 32 records>, <FluxTable: 8 columns, 418 records>, <FluxTable: 8 columns, 3 records>, <FluxTable: 8 columns, 1705 records>, <FluxTable: 8 columns, 1706 records>]
Done in 2.6381161212921143 seconds
0
No data found for bucket 2023-07-25 16:25:36 |> FS-jarama-fadrique


In [46]:
download_data('2023-10-22 12:07:52 |> FS-baja-version', CLOUD_URL, CLOUD_TOKEN, CLOUD_ORG)

Bucket 2023-10-22 12:07:52 |> FS-baja-version is not valid
